In [1]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import*
from datetime import datetime
from xgboost import XGBRegressor

In [369]:
data = {
    'tra': pd.read_csv('../input/J-league/train.csv'),
    'tes': pd.read_csv('../input/J-league/test.csv'),
    '14_ad': pd.read_csv('../input/J-league/2014_add.csv'),
    'cd': pd.read_csv('../input/J-league/condition.csv'),
    'cd_ad': pd.read_csv('../input/J-league/condition_add.csv'),
    'tr_ad': pd.read_csv('../input/J-league/train_add.csv'),
    'st': pd.read_csv('../input/J-league/stadium.csv'),
    'ss': pd.read_csv('../input/J-league/sample_submit.csv',header=None)
    }

In [370]:
data['tra']=data['tra'].append(data['tr_ad'])
data['tra']=data['tra'].sort_values('id').reset_index(drop=True)
data_2012=data['tra'][data['tra'].year==2012]
data_2013=data['tra'][data['tra'].year==2013].reset_index(drop=True)
data_2014=data['tra'][data['tra'].year==2014]
#2014年度データ結合
data_2014_all=data_2014.append(data['tes']).reset_index(drop=True)

C:\Users\miyak\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [371]:
#ホームでの試合回数の関数
def home_match_count(s):
    a=[]
    team=[]
    for i in range(len(s)):
        if True not in [j in s[i] for j in team]:
            team.append(s[i])
            a.append(1)
        else:
            team.append(s[i])
            a.append(team.count(s[i]))
    return a

In [372]:
a=home_match_count(data_2012['home'])
b=home_match_count(data_2013['home'])
c=home_match_count(data_2014_all['home'])

In [373]:
#home_match_countを全ての年に代入
home_match_count=pd.DataFrame(data=a).rename(columns={0: 'home_match_count'})
data_2012['home_match_count']=home_match_count
home_match_count=pd.DataFrame(data=b).rename(columns={0: 'home_match_count'})
data_2013['home_match_count']=home_match_count
home_match_count=pd.DataFrame(data=c).rename(columns={0: 'home_match_count'})
data_2014_all['home_match_count']=home_match_count

C:\Users\miyak\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [374]:
#2014年度データ再分割
data['tes']=data_2014_all[data_2014_all.y.isnull()].reset_index(drop=True)
data['tes']=data['tes'].drop('y', axis=1)
data_2014=data_2014_all[data_2014_all.y.isnull()==False]

In [375]:
data['tra']=data_2012.append(data_2013).sort_values('id').reset_index(drop=True)
cols=list(data['tra'].columns)
#data_2014のcolumnの並び替え
data_2014=data_2014[cols]
#trainデータリセット
data['tra']=data['tra'].append(data_2014).sort_values('id').reset_index(drop=True)
#来場者0人のデータを削除
data['tra']=data['tra'][data['tra'].y!=0].reset_index(drop=True)

In [376]:
unique_teams = data['tes']['home'].unique()
teams=pd.concat([pd.DataFrame({'home': unique_teams, 'home_match_count': [i]*len(unique_teams)}) for i in np.arange(1,22)], axis=0, ignore_index=True).reset_index(drop=True)
#デフォルトでグループラベルが index になる。index にしたく無い場合は as_index=False を指定する。
#ホームでの1試合目、2試合目ごとの平均(2012～2014)
tmp=data['tra'].groupby(['home','home_match_count'], as_index=False)['y'].mean().rename(columns={'y':'mean_y'})
teams = pd.merge(teams, tmp, how='left', on=['home','home_match_count'])

In [377]:
#home_match_count結合
data['tra']=pd.merge(data['tra'], teams, how='left', on=['home','home_match_count'])
data['tes']=pd.merge(data['tes'], teams, how='left', on=['home','home_match_count'])
cols_tes=list(data['tra'].columns)
cols_tes.remove('y')
data['tes']=data['tes'][cols_tes]

In [378]:
#前処理2
cols_all=list(data['tra'].columns)
all_data=pd.concat([data['tra'], data['tes']], axis=0).sort_values('id').reset_index(drop=True)
all_data=all_data[cols_all]

C:\Users\miyak\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [379]:
all_data['dow']=all_data['gameday'].apply(lambda x: str(x).split('(')[1])
all_data['hol']=all_data['dow'].apply(lambda x: x.replace(str(x),'1') if len(str(x))>2 else 0)
all_data['hol']=all_data['hol'].astype(int)
#day作成
all_data['day']=all_data['gameday'].apply(lambda x: str(x).split('(')[0])
all_data['day']=all_data['day'].map(lambda x: str(x).replace('/',''))
#year文字列変換
all_data['year']=all_data['year'].astype(str)
#data_int作成
all_data['data_int']=(all_data['year']+all_data['day']).astype(int)
cols_drop=['year','match','gameday','day']
all_data=all_data.drop(cols_drop, axis=1)
#timeは':'で分けて時のみをastype(int)
all_data['time']=all_data['time'].map(lambda x:str(x).split(':')[0]).astype(int)
#tvラベルエンコーディング
lbl = preprocessing.LabelEncoder()
for i in range(5):
    all_data['tv'+str(i)] = lbl.fit_transform(all_data['tv'].map(lambda x: str(str(x).split('／')[i]) if len(str(x).split('／'))>i else ''))
    all_data['tv'+str(i)] = lbl.fit_transform(all_data['tv'].map(lambda x: str(str(x).split('／')[i]) if len(str(x).split('／'))>i else ''))
all_data=all_data.drop('tv', axis=1)
all_data['home']=lbl.fit_transform(all_data['home'])
#ラベルエンコーディング
for i in ['stage','away','dow']:
    all_data[i]=lbl.fit_transform(all_data[i])

In [380]:
#特徴量2終了
all_data.shape

(2265, 17)

In [381]:
data['st']=data['st'].rename(columns={'name':'stadium'})
data['st']['prefecture']=data['st']['address'].apply(lambda x: re.split('[都県道府]', x)[0])
data['st']['city']=data['st']['address'].apply(lambda x: re.split('[都県市区]', x)[1]) 
data['st']=data['st'].drop('address',axis=1)
for i in ['prefecture','city']:
    data['st'][i]=lbl.fit_transform(data['st'][i])

In [382]:
#スタジアムデータ結合
all_data=pd.merge(all_data, data['st'], how='inner', on=['stadium']).sort_values('id').reset_index(drop=True)
all_data['stadium']=lbl.fit_transform(all_data['stadium'])

In [383]:
#data['cd'](コンディションデータ処理)
#ad追加
data['cd']=data['cd'].append(data['cd_ad']).sort_values('id').reset_index(drop=True)

H_member=['home_01','home_02','home_03','home_04','home_05','home_06','home_07','home_08'
         ,'home_09','home_10','home_11']

A_member=['away_01','away_02','away_03','away_04','away_05','away_06','away_07','away_08'
         ,'away_09','away_10','away_11']

#チーム名文字修正
data['cd']['home_team']=data['cd']['home_team'].map(lambda x : str(x).replace('・',''))
data['cd']['home_team']=data['cd']['home_team'].map(lambda x : str(x).replace('.',''))

#選手の苗字と名前のスペース削除
for i in H_member:
    data['cd'][i]=data['cd'][i].map(lambda x: str(x).replace('　',''))
    
for i in A_member:
    data['cd'][i]=data['cd'][i].map(lambda x: str(x).replace('　',''))

In [384]:
teams=data['cd']['home_team'].unique()
teams_member=data['cd']['home_team'].unique()

for i in range(len(H_member)):
    H_member[i]=data['cd'].groupby('home_team')[H_member[i]].unique()
    
for j in range(len(teams)):
    a=[]
    for i in range(len(H_member)):
        a.extend(H_member[i][teams[j]])
    s=pd.unique(a)
    teams_member[j]=s

#チームをラベルエンコーディング
team_list=teams.tolist()
data['cd']['home_team']=data['cd']['home_team'].map(lambda x: str(x).replace(str(x),str(team_list.index(str(x)))))
data['cd']['home_team']=data['cd']['home_team'].astype(int)

In [385]:
def member_num(a):
    s=[]
    for i in range(len(data['cd'])):
        s.append(teams_member[data['cd']['home_team'][i]].tolist().index(data['cd'][a][i]))
    return s

home_01=member_num('home_01')
home_02=member_num('home_02')
home_03=member_num('home_03')
home_04=member_num('home_04')
home_05=member_num('home_05')
home_06=member_num('home_06')
home_07=member_num('home_07')
home_08=member_num('home_08')
home_09=member_num('home_09')
home_10=member_num('home_10')
home_11=member_num('home_11')

home=['home_01','home_02','home_03','home_04','home_05','home_06','home_07','home_08'
     ,'home_09','home_10','home_11']
data['cd']=data['cd'].drop(home,axis=1)

home_member=pd.DataFrame({'home_01':home_01,'home_02':home_02,'home_03':home_03,'home_04':home_04,'home_05':home_05,'home_06':home_06,'home_07':home_07,'home_08':home_08,
             'home_09':home_09,'home_10':home_10,'home_11':home_11})

data['cd']=pd.concat([data['cd'],home_member], axis=1)
#home_teamエンコーディング終了

In [386]:
#アウェーチーム
a_teams=data['cd']['away_team'].unique()
a_teams_member=data['cd']['away_team'].unique()

for i in range(len(A_member)):
    A_member[i]=data['cd'].groupby('away_team')[A_member[i]].unique()
    
for j in range(len(a_teams)):
    a=[]
    for i in range(len(A_member)):
        a.extend(A_member[i][a_teams[j]])
    s=pd.unique(a)
    a_teams_member[j]=s

#チームをラベルエンコーディング
a_team_list=a_teams.tolist()
data['cd']['away_team']=data['cd']['away_team'].map(lambda x: str(x).replace(str(x),str(a_team_list.index(str(x)))))
data['cd']['away_team']=data['cd']['away_team'].astype(int)

In [387]:
def a_member_num(a):
    s=[]
    for i in range(len(data['cd'])):
        s.append(a_teams_member[data['cd']['away_team'][i]].tolist().index(data['cd'][a][i]))
    return s

away_01=a_member_num('away_01')
away_02=a_member_num('away_02')
away_03=a_member_num('away_03')
away_04=a_member_num('away_04')
away_05=a_member_num('away_05')
away_06=a_member_num('away_06')
away_07=a_member_num('away_07')
away_08=a_member_num('away_08')
away_09=a_member_num('away_09')
away_10=a_member_num('away_10')
away_11=a_member_num('away_11')

away=['away_01','away_02','away_03','away_04','away_05','away_06','away_07','away_08'
     ,'away_09','away_10','away_11']
data['cd']=data['cd'].drop(away,axis=1)

away_member=pd.DataFrame({'away_01':away_01,'away_02':away_02,'away_03':away_03,'away_04':away_04,'away_05':away_05,'away_06':away_06,'away_07':away_07,'away_08':away_08,
             'away_09':away_09,'away_10':away_10,'away_11':away_11})

data['cd']=pd.concat([data['cd'],away_member], axis=1)
#away_teamエンコーディング終了

In [388]:
#その他ラベルエンコーディング
data['cd']['humidity']=data['cd']['humidity'].map(lambda x: str(x).replace('%',''))
data['cd']['humidity']=data['cd']['humidity'].astype(int)
lbl = preprocessing.LabelEncoder()
for i in ['weather','referee']:
    data['cd'][i]=lbl.fit_transform(data['cd'][i])

In [389]:
#data['cd']終わり
data['cd'].head()

,id,home_score,away_score,weather,temperature,humidity,referee,home_team,away_team,home_01,...,away_02,away_03,away_04,away_05,away_06,away_07,away_08,away_09,away_10,away_11
0,13994,1,0,26,3.8,66,35,0,0,0,...,1,5,8,10,3,14,15,25,30,23
1,13995,1,0,0,12.4,43,48,1,1,0,...,4,13,7,16,14,5,25,29,23,26
2,13996,2,3,8,11.3,41,52,2,2,0,...,3,13,14,15,25,21,33,23,28,26
3,13997,1,0,20,11.4,52,38,3,3,0,...,3,6,10,11,15,19,12,17,23,27
4,13998,0,0,0,22.5,32,30,4,4,0,...,3,11,15,14,19,24,26,31,30,36


In [390]:
all_data=pd.merge(all_data, data['cd'], how='inner', on=['id']).sort_values('id').reset_index(drop=True)

In [391]:
test_p=all_data[all_data.y.isnull()].reset_index(drop=True)
test=test_p.drop(['id','y'],axis=1)

In [392]:
train_p=all_data[all_data.y.isnull()==False]
train_x=train_p.drop(['id','y'],axis=1)
train_y=train_p['y'].values

In [393]:
test=test.fillna(-1)
train_x=train_x.fillna(-1)

In [394]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [395]:
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.1, random_state=3, n_estimators=150, subsample=0.8, 
                      max_depth =3)
model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
model3 = XGBRegressor(learning_rate=0.1, random_state=3, n_estimators=230, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =3)

In [396]:
model1.fit(train_x, np.log1p(train_y))
model2.fit(train_x, np.log1p(train_y))
model3.fit(train_x, np.log1p(train_y))

preds1 = model1.predict(train_x)
preds2 = model2.predict(train_x)
preds3 = model3.predict(train_x)

In [397]:
print('RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(train_y), preds1))
print('RMSE KNeighborsRegressor: ', RMSLE(np.log1p(train_y), preds2))
print('RMSE XGBRegressor: ', RMSLE(np.log1p(train_y), preds3))

RMSE GradientBoostingRegressor:  0.16526120980960385
RMSE KNeighborsRegressor:  0.21066092198735972
RMSE XGBRegressor:  0.15028585602530076


In [398]:
preds1 = model1.predict(test)
preds2 = model2.predict(test)
preds3 = model3.predict(test)

In [399]:
ans = 0.3*preds1+0.3*preds2+0.4*preds3

In [400]:
ans = np.expm1(ans)

In [401]:
data['ss'].shape

(313, 2)

In [402]:
data['ss']=data['ss'].drop([1],axis=1)

In [403]:
data['ss']=data['ss'].values.flatten()

In [404]:
df_result=pd.DataFrame({0:data['ss'],1:ans})

In [405]:
df_result.to_csv("sample_submit5.csv",index=False,header=False)

In [406]:
#最終結果
#スコア4,154
#672位/943位
#明らかに過学習している。
#mean_yはあまり結果に依存していないことが分かった。
#パラメーターを変更してもそこまで変化が無かった
df_result.head()

,0,1
0,15822,13251.693829
1,15823,16055.461378
2,15824,39839.191639
3,15825,11098.533922
4,15827,25333.105574
